In [1]:
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time

import datasets.mvtec as mvtec

from feature_extraction import Resnet18Features, WideResnet50Features, extractEmbeddingVectors, extractEmbeddingVectorsDataloader, getIndices
from distribution_fitting import getMeanCov
from score_calculation import calculatePatchScore, calculateImageScore, calculateImageClassification, calculatePatchClassification
# from padim_utils import saveFeatures

# from torchvision.models import wide_resnet50_2, resnet18


In [2]:
use_cuda = torch.cuda.is_available()
# device = torch.device('cuda' if use_cuda else 'cpu')
device = torch.device('cpu')
device

device(type='cpu')

In [3]:
class_name = 'up'
data_path = '../data/aligned_pucks/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
len(dataset)
# x = next(iter(dataloader))[0][0].unsqueeze(0)
# x.shape

100

In [4]:
features_model = Resnet18Features(device)
# features_model = WideResnet50Features(device)

In [5]:
tic = time.perf_counter()
embedding_vectors = extractEmbeddingVectorsDataloader(features_model, dataloader, device)
print(time.perf_counter()-tic)

Feature extraction:   0%|          | 0/4 [00:00<?, ?it/s]/home/plejd/anaconda3/envs/vision/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Feature extraction: 100%|██████████| 4/4 [00:07<00:00,  1.76s/it]

7.063172661000863


In [6]:
idx = getIndices(100, 448, device)  
# idx = getIndices(550, 1792, device)  
# idx = getIndices(400, 1792, device)  
embedding_vectors = torch.index_select(embedding_vectors, 1, idx)


In [7]:
embedding_vectors.shape

torch.Size([100, 100, 56, 56])

In [8]:
tic = time.perf_counter()
mean, cov, cov_inv = getMeanCov(embedding_vectors, torch.device('cpu'))
print(time.perf_counter()-tic)

Calculating covariance:   4%|▍         | 118/3136 [00:00<00:02, 1094.41it/s]

Calculating mean


Calculating covariance: 100%|██████████| 3136/3136 [00:02<00:00, 1227.89it/s]


Calculating inverse of covariance
3.221637168957386


In [9]:
torch.save(mean, './distributions/up_mean_test2.pt')
# torch.save(cov, './distributions/up_cov_test.pkl')
torch.save(cov_inv, './distributions/up_cov_inv_test2.pt')
